#### Brian Ramirez

# Market Basket Analysis

## Business Question
<p>In a data set of patient prescriptions, can Market Basket Analysis (MBA) be used to identify which medications are commonly prescribed together? Identifying prescription trends can help the organization keep an pharmacy properly stocked to increase patient satisfaction.</p>

## Goal
<p>The goal of the data analysis is to use Market Basket Analysis to identify which prescriptions are commonly prescribed together.</p>

## Market Basket Analysis Justification
<p>Market Basket Analysis is an effective tool for finding underlying connections between variables. According to <em>Toward Data Science</em>, MBA works by "looking for combinations of items that occur together frequently in transactions"(Li, 2017). Finding reoccurring combinations can help data analysts identify trends.</p>

### Example of a Transaction in the Data Set
<p>Each set of medications prescribed to a patient is a transaction. An example of a transaction comes from the third record in the dataset:<br>
    <em>Citalopram, Benicar, Amphetamine Salt Combo XR.</em><br>
The set of medications above were prescribed to a patient as a single transaction.</p>

## Assumption of Market Basket Analysis
<p>Market Basket Analysis assumes that if two items frequently appear in the same set, they must have an association.</p>

## Data Preparation
### Import Packages and Libraries

<p>The following packages and libraries will be imported for the analysis:</p>

|Import|Justification|
|:---|:---|
|apriori| To perform the Apriori Algorithm|
|association_rules|To create the association rules|
|filterwarnings|To filter and ignore redundant error messages.|
|pandas|To create and manipulate dataframes.|
|TransactionEncoder|To transactionalize the dataset before performing the Apriori Algorithm.|

In [1]:
# Import packages and libraries

import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import (association_rules, apriori)

import warnings
warnings.filterwarnings('ignore')

<strong>Load the CSV file and create a DataFrame</strong>

In [2]:
# Load the csv file into Jupyter Notebook and create a DataFrame

df = pd.read_csv('medical_market_basket.csv')

In [3]:
# Display DataFrame info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Presc01  7501 non-null   object
 1   Presc02  5747 non-null   object
 2   Presc03  4389 non-null   object
 3   Presc04  3345 non-null   object
 4   Presc05  2529 non-null   object
 5   Presc06  1864 non-null   object
 6   Presc07  1369 non-null   object
 7   Presc08  981 non-null    object
 8   Presc09  654 non-null    object
 9   Presc10  395 non-null    object
 10  Presc11  256 non-null    object
 11  Presc12  154 non-null    object
 12  Presc13  87 non-null     object
 13  Presc14  47 non-null     object
 14  Presc15  25 non-null     object
 15  Presc16  8 non-null      object
 16  Presc17  4 non-null      object
 17  Presc18  4 non-null      object
 18  Presc19  3 non-null      object
 19  Presc20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


In [4]:
# Display the DataFrame

df.head()

,Presc01,Presc02,Presc03,Presc04,Presc05,Presc06,Presc07,Presc08,Presc09,Presc10,Presc11,Presc12,Presc13,Presc14,Presc15,Presc16,Presc17,Presc18,Presc19,Presc20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amlodipine,albuterol aerosol,allopurinol,pantoprazole,lorazepam,omeprazole,mometasone,fluconozole,gabapentin,pravastatin,cialis,losartan,metoprolol succinate XL,sulfamethoxazole,abilify,spironolactone,albuterol HFA,levofloxacin,promethazine,glipizide
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,citalopram,benicar,amphetamine salt combo xr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Display data types

print(df.dtypes)

Presc01    object
Presc02    object
Presc03    object
Presc04    object
Presc05    object
Presc06    object
Presc07    object
Presc08    object
Presc09    object
Presc10    object
Presc11    object
Presc12    object
Presc13    object
Presc14    object
Presc15    object
Presc16    object
Presc17    object
Presc18    object
Presc19    object
Presc20    object
dtype: object


In [6]:
# Display dimensions

print(df.shape)
print(len(df))

(15002, 20)
15002


### Remove Rows with Null Values
7501 rows have null values and will be removed.

In [7]:
df.isnull().all(axis = 1).sum()

7501

In [8]:
df = df.dropna(how = 'all')
df.isnull().all(axis = 1).sum()

0

In [9]:
print(df.shape)
print(len(df))

(7501, 20)
7501


### Transactionalize the Data Frame
<p>The dataset must be transactionalized to be suitable for the <em>Apriori</em> Function.<br>
The dataframe will first be converted to a list and then encoded.</p>

In [10]:
# Convert DataFrame into a list

med_list = []

for i in range(0, 7501):
    med_list.append([str(df.values[i,j]) for j in range(0, 20)])

In [11]:
# Encode the dataframe

encoder = TransactionEncoder()
med_array = encoder.fit(med_list).transform(med_list)

In [12]:
# Display encoded DataFrame

clean_df = pd.DataFrame(med_array, columns = encoder.columns_)
clean_df

,Duloxetine,Premarin,Yaz,abilify,acetaminophen,actonel,albuterol HFA,albuterol aerosol,alendronate,allopurinol,...,trazodone HCI,triamcinolone Ace topical,triamterene,trimethoprim DS,valaciclovir,valsartan,venlafaxine XR,verapamil SR,viagra,zolpidem
0,False,False,False,True,False,False,True,True,False,True,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Check for Null Columns
<p>Any columns marked as <em>"nan"</em> will be dropped before performing the Apriori Function.</p>

In [13]:
for col in clean_df.columns:
    print(col)

Duloxetine
Premarin
Yaz
abilify
acetaminophen
actonel
albuterol HFA
albuterol aerosol
alendronate
allopurinol
alprazolam
amitriptyline
amlodipine
amoxicillin
amphetamine
amphetamine salt combo
amphetamine salt combo xr
atenolol
atorvastatin
azithromycin
benazepril
benicar
boniva
bupropion sr
carisoprodol
carvedilol
cefdinir
celebrex
celecoxib
cephalexin
cialis
ciprofloxacin
citalopram
clavulanate K+
clonazepam
clonidine HCI
clopidogrel
clotrimazole
codeine
crestor
cyclobenzaprine
cymbalta
dextroamphetamine XR
diazepam
diclofenac sodium
doxycycline hyclate
enalapril
escitalopram
esomeprazole
ezetimibe
fenofibrate
fexofenadine
finasteride
flovent hfa 110mcg inhaler
fluconozole
fluoxetine HCI
fluticasone
fluticasone nasal spray
folic acid
furosemide
gabapentin
glimepiride
glipizide
glyburide
hydrochlorothiazide
hydrocodone
hydrocortisone 2.5% cream
ibuprophen
isosorbide mononitrate
lansoprazole
lantus
levofloxacin
levothyroxine sodium
lisinopril
lorazepam
losartan
lovastatin
meloxicam
met

<p>A <em>"nan"</em> column can be found between the <em>"mometasone"</em> and<em>"naproxen"</em> columns.<br>
The <em>"nan"</em> will be dropped before continuing.</p>

In [14]:
# Remove the "nan" column.

med_df = clean_df.drop(['nan'], axis = 1)

In [15]:
# Display columns with "nan" removed.

for col in med_df.columns:
    print(col)

Duloxetine
Premarin
Yaz
abilify
acetaminophen
actonel
albuterol HFA
albuterol aerosol
alendronate
allopurinol
alprazolam
amitriptyline
amlodipine
amoxicillin
amphetamine
amphetamine salt combo
amphetamine salt combo xr
atenolol
atorvastatin
azithromycin
benazepril
benicar
boniva
bupropion sr
carisoprodol
carvedilol
cefdinir
celebrex
celecoxib
cephalexin
cialis
ciprofloxacin
citalopram
clavulanate K+
clonazepam
clonidine HCI
clopidogrel
clotrimazole
codeine
crestor
cyclobenzaprine
cymbalta
dextroamphetamine XR
diazepam
diclofenac sodium
doxycycline hyclate
enalapril
escitalopram
esomeprazole
ezetimibe
fenofibrate
fexofenadine
finasteride
flovent hfa 110mcg inhaler
fluconozole
fluoxetine HCI
fluticasone
fluticasone nasal spray
folic acid
furosemide
gabapentin
glimepiride
glipizide
glyburide
hydrochlorothiazide
hydrocodone
hydrocortisone 2.5% cream
ibuprophen
isosorbide mononitrate
lansoprazole
lantus
levofloxacin
levothyroxine sodium
lisinopril
lorazepam
losartan
lovastatin
meloxicam
met

<p>The <em>"nan"</em> column has been removed.</p>

#### Copy of the Cleaned Data Set

In [16]:
# Display data dimensions

med_df.shape

(7501, 119)

In [17]:
# Create a copy of the dataset

med_df.to_csv('T3_clean_df')

### Generate Association Rules with the Apriori Agorithm

In [18]:
# Calculate frequent items

frq_items = apriori(med_df, min_support = 0.05, use_colnames = True)

print('Frequent Items:')
frq_items.head()

Frequent Items:


,support,itemsets
0,0.238368,(abilify)
1,0.079323,(alprazolam)
2,0.071457,(amlodipine)
3,0.068391,(amphetamine salt combo)
4,0.179709,(amphetamine salt combo xr)


In [19]:
# Generate association rules table

print('Association Rules Table')
rules = association_rules(frq_items, metric = 'lift', min_threshold = 1)
rules.head()

Association Rules Table


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(abilify),(amphetamine salt combo xr),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562
1,(amphetamine salt combo xr),(abilify),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
2,(abilify),(carvedilol),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
3,(carvedilol),(abilify),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
4,(abilify),(diazepam),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197


### Support, Lift, and Confidence Values

<p>The support, lift, and confidence of the association rules table are displayed below.</p>

In [20]:
# Display association rules table values

print('Association Rules Table Values:')
association_values = (rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']])
association_values

Association Rules Table Values:


,antecedents,consequents,support,lift,confidence
0,(abilify),(amphetamine salt combo xr),0.050927,1.188845,0.213647
1,(amphetamine salt combo xr),(abilify),0.050927,1.188845,0.283383
2,(abilify),(carvedilol),0.059725,1.439085,0.250559
3,(carvedilol),(abilify),0.059725,1.439085,0.343032
4,(abilify),(diazepam),0.052660,1.348332,0.220917
5,(diazepam),(abilify),0.052660,1.348332,0.321400


### Top Three Rules Generated by the Apriori Algorithm

<strong>Top Three Rules by Lift</strong>

In [21]:
# Generate top rule by Lift

print('Top 3 Rules by Lift:')
top_lift = rules.sort_values(['lift'], ascending = False)
top_lift.head(3)

Top 3 Rules by Lift:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(abilify),(carvedilol),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
3,(carvedilol),(abilify),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
5,(diazepam),(abilify),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965


## Data Summary and Implications

### Significance of the Support, Lift, and Confidence
The top three rules suggest the following:<br>
<ul>
    <li>25% confidence that a prescription of Abilify is 1.44 times likely to lead to a prescription of Carvedilol.</li>
    <li>34% confidence that a prescription of Carvedilol is 1.44 times likely to lead to a prescription of Abilify.</li>
    <li>32% confidence that a prescription of Diazepam is 1.35 times likely to lead to a prescription of Abilify.</li>
</ul>

#### Support
<p>The Support column is significant because it refers to the frequency of an item set.<br>
    Hospitals within the organization should keep a steady supply of frequently prescribed item sets stocked to prevent patients from missing a dose. Support is significant because the algorithm has a minimum support of 0.05, so only item sets that appear in the data set at least 5% of the time are captured.</p>

#### Lift
<p>The Lift column is significant because it refers to the probability of one item leading to the occurrence of another.<br>
    Hospitals within the organization should plan on stocking frequently paired prescriptions to prevent a missed dose during a patient's stay.</p>

#### Confidence
<p>The Confidence column is significant because it refers to how likely a consequent is to appear in a set when the set has the corresponding antecedent.<br>
The antecedents and consequents in the top three rules are as follows:</p>

|Antecedant|Consequent|
|:--|:--|
|Abilify|Carvedilol|
|Carvedilol|Abilify|
|Diazepam|Abilify|

### Practical Significance of the Findings
<p>The findings from the analysis provide insight into medications that are frequently prescribed together. The insight is significant because the hospital can plan to keep certain medications stocked if they frequently appear together in a set.<br>
    Keeping frequently paired medications stocked will ensure that patients do not miss a dose during an initial stay. Staff can also use the stocked medications to provide patients with take-home prescriptions if necessary.<br>
Ensuring patients have access to required medications is a medical necessity that could prevent further fines from Centers for Medicare and Medicaid Services.</p>

### Recommended Course of Action
<p>The organization should plan to order certain medications based on their frequency of occurrence within a given set.<br>
Based on the results of the analysis, the following medications should be ordered in sets:</p>
<ul>
    <li>Abilify and Carvedilol</li>
    <li>Carvedilol and Abilify</li>
    <li>Diazepam and Abilify</li>
</ul>
<p>Keeping frequently paired prescriptions stocked will ensure patients have access to required medication during their stay.</p>

#### References:
<p>Li, Susan. 2017. Toward Data Science. <em>A Gentle Introduction on Market Basket Analysis.</em> <br>https://towardsdatascience.com/a-gentle-introduction-on-market-basket-analysis-association-rules</p>